In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
import shutil #kopyalama için kullanıyoruz
from sklearn.model_selection import train_test_split
import glob     #dosya okuma fonksiyonları
import random   #dosya okuma fonksiyonları 
import os       #dosya okuma fonksiyonları

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import time
import datetime
import sys

from torch.autograd import Variable
import torch
import numpy as np
import itertools
from tqdm.notebook import tqdm      #eğitim aşamasında nerde olduğumuzu görmek için kullanıyoruz
"""TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(range(epoch, n_epochs))"""


import matplotlib.pyplot as plt
import matplotlib.animation  as animation
from IPython.display import HTML , Image as ImgDisplay # burada sorun çıkabilir çıkarsa from IPython.display import Image as ImgDisplay olacak şekilde aşağıya yaz 
from torchsummary import summary
from torchvision.utils import save_image
import pandas as pd

In [ ]:
image_dir = "./img_align_celeba/img_align_celeba"
attributes_file = "./list_attr_celeba.csv"
output_dir = "./preprocessed_dataset_celeba"

data = pd.read_csv(attributes_file)

In [ ]:
# Siyah saç özelliği "1" olan Kadın ünlülerden 1000 tane örnek seçtik, dataframe'e attık.
df_blackHair = data.loc[data["Black_Hair"] == 1 & (data["Male"] == -1)].sample(n = 1000)

#Sarı saç özelliği "1" olan Kadın ünlülerden 1000 tane örnek seçtik, dataframe'e attık.
df_blonde = data.loc[data["Blond_Hair"] == 1 & (data["Male"] == -1)].sample(n = 1000)

In [ ]:
#Seçilen resimlerin isimlerine daha rahat ulaşmak için bir listeye kaydedelim
domainK, domainD = [] , []

for index, row in df_blackHair.iterrows():
    domainK.append(row["image_id"])

for index, row in df_blonde.iterrows():
    domainD.append(row["image_id"])

In [ ]:
# Train ve test verilerini ilk seçtiğimiz 1000'er resimden bölerek oluşturalım.
K_train, K_test = train_test_split(domainK, test_size = 0.01, random_state = 1453 )
D_train, D_test = train_test_split(domainD, test_size = 0.01, random_state = 1453 )

Eğitim ve test veri seti başka bir yere kaydediliyor

In [ ]:
N = min(len(K_train), len(D_train))
K_train = K_train[:N]
D_train = D_train[:N]
print(f"Images in K(black) {len(K_train)} and D(blonde) {len(D_train)}")
K_train_dir = os.path.join(output_dir, "train/K")
D_train_dir = os.path.join(output_dir, "train/D")

os.makedirs(K_train_dir, exist_ok = True)
os.makedirs(D_train_dir, exist_ok = True)

for imageK, imageD in zip(K_train, D_train):
    shutil.copy(os.path.join(image_dir, imageK), os.path.join(K_train_dir, imageK))
    shutil.copy(os.path.join(image_dir, imageD), os.path.join(D_train_dir, imageD))

#test verisi de eşit olsun
N = min(len(K_test), len(D_test))
K_test = K_test[:N]
D_test = D_test[:N]
print(f"Images in K(black) {len(K_test)} and D(blonde) {len(D_test)}")

K_test_dir = os.path.join(output_dir, "test/K")
D_test_dir = os.path.join(output_dir, "test_D")

os.makedirs(K_test_dir, exist_ok = True)
os.makedirs(D_test_dir, exist_ok = True)

for imageK, imageD in zip(K_test, D_test):
    shutil.copy(os.path.join(image_dir, imageK), os.path.join(K_test_dir, imageK))
    shutil.copy(os.path.join(image_dir, imageD), os.path.join(D_test_dir, imageD))

DATASET SINIFIMIZ

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_ = None, unaligned = False, mode = "train"):    # root = verinin bulunduğu kök klasör
        #gelen transform fonksiyonu kullanılıyoe
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_K = sorted(glob.glob(os.path.join(root, "%s/K" %mode) + "/*.*"))        
        self.files_D = sorted(glob.glob(os.path.join(root, "%s/D" %mode) + "/*.*"))

    def __getitem__(self,index):        #eğitim yapılırken her veri alışında bu fonksiyon çalışır yani
        #eğitim yapılırken her veri alışında bu fonksiyona giriliyor ve veri ön işlemeden geçiriliyor
        item_K = self.transform(Image.open(self.files_K[index % len(self.files_K)]))

        if self.unaligned:
            # veriler eşli olmadığından dolayı rastgele bir foto seçiliyo
            item_D = self.transform(Image.open(self.files_D[random.randint(0, len(self.files_D) - 1)]))
        else:
            item_D = self.transform(Image.open(self.files_D[index % len(self.files_D)]))

        return {"K(black)": item_K, "D(blonde)": item_D}

    def __len__(self):
        return max(len(self.files_K), len(self.files_D))

GENERATOR BLOKLARI VE GENERATOR YAPAY SİNİR AĞI TASARIMI

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features,in_features,3),
            nn.ReLU(inplace = True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features,in_features,3),
            nn.InstanceNorm2d(in_features)
        ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks = 9):
        super(Generator, self).__init__()

        #Initial convolution block
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace = True)
        ]

        #Downsampling
        in_features = 64
        out_features = in_features*2
        for _ in range(2):
            model += [
                  nn.Conv2d(in_features,out_features,3, stride = 2, padding = 1),
                  nn.InstanceNorm2d(out_features),
                  nn.ReLU(inplace = True)
            ]
            in_features = out_features
            out_features = in_features*2

        #Residual Blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        #Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [
                  nn.ConvTranspose2d(in_features,out_features,3, stride = 2, padding = 1, output_padding = 1),
                  nn.InstanceNorm2d(out_features),
                  nn.ReLU(inplace = True)
            ]
            in_features = out_features
            out_features = in_features//2

        #Output LAyer
        model += [
                  nn.ReflectionPad2d(3),
                  nn.Conv2d(64, output_nc, 7),
                  nn.Tanh()
            ]
        
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

Discriminator Yapay Sinir Ağı Tasarımı

In [ ]:
class Discriminator(nn.Module):
    
    def __init__(self, input_nc):
        #PatchGAN discriminatorunun yapısı
        super(Discriminator, self).__init__()


        #A brunch of convolutions one after another
        model = [
            nn.Conv2d(input_nc, 64 ,4 , stride= 2, padding= 1),
            nn.LeakyReLU(0.2, inplace= True)
        ]

        model += [
            nn.Conv2d(64 ,128, 4, stride= 2, padding= 1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace= True)
        ]
    
        model += [
            nn.Conv2d(128, 256, 4, stride= 2, padding= 1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace= True)
        ]
    
        model += [
            nn.Conv2d(256, 512, 4, stride= 2, padding= 1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace= True)
        ]
    
        #FCN classification Layer
        model += [nn.Conv2d(512, 1, 4, padding= 1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        x = self.model(x)
        #Average pooling and flatten
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)
    
    

EK FONKSİYONLAR

In [ ]:
def tensor2image(tensor):
    #görselleştirmek için gpu'ya atılmış tensor verisi işlemci üzerinde imgeye dönüştürülür.
    image = 127.5*(tensor[0].cpu().float().numpy() + 1.0)
    if image.shape[0] == 1:
        image = nn.tile(image, (3,1,1))
    return image.astype(np.uint8)

class ReplayBuffer():
    #Kayıp hesabı yapılırken Buffer a veri atıp çıkartıyoruz
    def __init__(self, max_size = 50):
        assert (max_size > 0), "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data :
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                to_return.append(element)
        return Variable(torch.cat(to_return))
    
class LambdaLR():
    #Learning rate decay ayarları burda yapılıyo
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0 ), "Decay must start before the trainin session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch, self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)
    
def weights_init_normal(m):
    #Yapay sinir ağının ağırlıklarının uniform bir dağılımda ilklendirilmesi için bu fonksiyon kullanılıyor.
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

Eğitim parametreleri

In [ ]:
epoch = 0
n_epochs = 50
batchSize = 1
dataraoot = "./preprocessed_dataset_celeba"
lr = 0.0002
decay_epoch = 3
size = 256
input_nc = 3
output_nc = 3
cuda = True
n_cpu = 8

In [ ]:
if torch.cuda.is_available() and not cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")


##### Definition of variables #####
#Networks
netG_K2D = Generator(input_nc, output_nc)
netG_D2K = Generator(output_nc, input_nc)
netD_K = Discriminator(input_nc)
netD_D = Discriminator(output_nc)

if cuda :
     netG_D2K.cuda()
     netG_K2D.cuda()
     netD_K.cuda()
     netD_D.cuda()

netG_K2D.apply(weights_init_normal)
netG_D2K.apply(weights_init_normal)
netD_K.apply(weights_init_normal)
netD_D.apply(weights_init_normal)

#Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()


#Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_K2D.parameters(), netG_D2K.parameters()),
                               lr = lr, betas = (0.5, 0.999))
optimizerD_K = torch.optim.Adam(netD_K.parameters(), lr = lr, betas = (0.5, 0.999))
optimizerD_D = torch.optim.Adam(netD_D.parameters(), lr = lr, betas = (0.5, 0.999))

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda = LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_schedulerD_K = torch.optim.lr_scheduler.LambdaLR(optimizerD_K, lr_lambda = LambdaLR(n_epochs, epoch, decay_epoch).step)
lr_schedulerD_D = torch.optim.lr_scheduler.LambdaLR(optimizerD_D, lr_lambda = LambdaLR(n_epochs, epoch, decay_epoch).step)

#Inputs & targets memory alloc
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
input_K = Tensor(batchSize, input_nc, size, size)
input_D = Tensor(batchSize, input_nc, size, size)
target_real = Variable(Tensor(batchSize,1). fill_(1.0), requires_grad = False)
target_fake = Variable(Tensor(batchSize,1). fill_(0.0), requires_grad = False)

fake_K_buffer = ReplayBuffer()
fake_D_buffer = ReplayBuffer()

#Dataset Loader
transforms_ = [
     transforms.Resize(int(size*1.12), Image.BICUBIC),
     transforms.RandomCrop(size),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]

dataloader = DataLoader(ImageDataset(dataraoot, transforms_=transforms_, unaligned= True),
                        batch_size = batchSize, shuffle = True, num_workers = n_cpu)

G_loss = []
G_identity_loss = []
G_gan_loss = []
G_cycle_loss = []
D_loss = []

In [ ]:
##### Training #####
pbar = tqdm(range(epoch, n_epochs))
for epoch in pbar :
    for i, batch in enumerate(dataloader):
        #Set model input
        real_K = Variable(input_K.copy_(batch["K"]))
        real_D = Variable(input_D.copy_(batch["D"]))

        ##### Generators K2D and D2K #####
        optimizer_G.zero_grad()

        #Identity loss
        #G_K2D(D) should D if real D is fed
        same_D = netG_K2D(real_K)
        loss_identity_D = criterion_identity(same_D, real_D)*5.0
        #G_D2K(K) should K if real K is fed
        same_K = netG_D2K(real_D)
        loss_identity_K = criterion_identity(same_K, real_K)*5.0

        #GAN Loss
        fake_D = netG_K2D(real_K)
        pred_fake = netD_D(fake_D)
        loss_GAN_K2D = criterion_GAN(pred_fake, target_real)
        
        fake_K = netG_D2K(real_D)
        pred_fake = netD_K(fake_K)
        loss_GAN_D2K = criterion_GAN(pred_fake, target_real)

        #Cycle Loss
        recovered_K = netG_D2K(fake_D)
        loss_cycle_KDK = criterion_cycle(recovered_K, real_K)*10.0

        recovered_D = netG_K2D(fake_K)        
        loss_cycle_DKD = criterion_cycle(recovered_D, real_D)*10.0

        #Total loss
        loss_G = loss_identity_K + loss_identity_D + loss_GAN_D2K + loss_GAN_K2D + loss_cycle_KDK + loss_cycle_DKD
        loss_G.backward()

        optimizer_G.step()

        ################################################

        ######## Discriminator K ########
        optimizerD_K.zero_grad()

        #Real loss
        pred_real = netD_K(real_K)
        loss_D_real = criterion_GAN(pred_real, target_real)

        #Fake loss
        fake_K = fake_K_buffer.push_and_pop(fake_K)
        pred_fake = netD_K(fake_K.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        #Total loss
        loss_D_K = (loss_D_real + loss_D_fake)*0.5
        loss_D_K.backward()

        optimizerD_K.step()

        ################################################

        ######## Discriminator D ########
        optimizerD_D.zero_grad()

        #Real loss
        pred_real = netD_D(real_D)
        loss_D_real = criterion_GAN(pred_real, target_real)

        #Fake loss
        fake_D = fake_D_buffer.push_and_pop(fake_D)
        pred_fake = netD_D(fake_D.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        #Total loss
        loss_D_D = (loss_D_real + loss_D_fake)*0.5
        loss_D_D.backward()

        optimizerD_D.step()

        ################################################        


        pbar.set_postfix({"loss_G": loss_G.item(),
                          "loss_G_cycle": (loss_cycle_KDK .item() + loss_cycle_DKD.item()), 
                          "loss_D":(loss_D_K.item() + loss_D_D.item())})
        G_loss.append(loss_G.item())
        G_identity_loss.append(loss_identity_K.item() + loss_identity_D.item())
        G_gan_loss.append(loss_GAN_K2D.item() + loss_GAN_D2K.item())
        G_cycle_loss.append(loss_cycle_KDK.item() + loss_cycle_DKD.item())
        D_loss.append(loss_D_K.item() + loss_D_D.item())

    # Update learning rates
    lr_scheduler_G.step()
    lr_schedulerD_K.step()
    lr_schedulerD_D.step()

    #Save models checkpoints
    torch.save(netG_K2D.state_dict(), "output/netG_K2D.pth")
    torch.save(netG_D2K.state_dict(), "output/netG_D2K.pth")
    torch.save(netD_K.state_dict(), "output/netD_K.pth")
    torch.save(netD_D.state_dict(), "output/netD_D.pth")

################################################